In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np

## Links:
* [Colah blog (LSTM/GRU)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
* [PyTorch tutorial - RNN for name classification](https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)
* [MNIST classification with RNN tutorial](https://medium.com/dair-ai/building-rnns-is-fun-with-pytorch-and-google-colab-3903ea9a3a79)
* [Good tutorials about Torch sentiment](https://github.com/bentrevett/pytorch-sentiment-analysis)

## Vanilla RNN

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png" width="600">

$$\Large h_{i+1} = tanh(W_x \cdot X_{i+1} + W_y \cdot h_{i})$$

# RNN intro


Давайте разберемся что из себя вообще представляют рекуррентные нейронные сети в самом простом виде.

В самом простом виде для одного входного вектора $x_{(t)}$ и одного слоя рекуррентной сети справедливо такое соотношение:

$$y_{(t)} = \phi (x_{(t)}^T \cdot w_x + y_{(t-1)}^T \cdot w_y + b)$$

где 
* $x(t)$ -- входной вектор на текущем шаге
* $y(t)$ -- выходной вектор на текущем шаге
* $w_x$ -- вектор весов нейронов для входа
* $w_y$ -- вектор весов нейронов для выхода
* $y(t-1)$ -- выходной вектор с прошлого шага. Для шага 0 этот вектор нулевой
* $b$ -- байес (bias)
* $\phi$ -- обозначение для функции активации, например ReLU


<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png" width="600">

Есть понятие **hidden_state** ( $h(t)$ ) -- это "память" рекуррентной ячейки.

В общем случае $h_{(t)} = f(h_{(t-1)}, x_{(t)})$, на выход также $y{(t)} = f(h{(t-1)}, x{(t)})$.


### Упрощение формулы

Снова немножко математики чтобы привести формулу выше к более удобному виду.

Представим, что на вход подается не один вектор $x_{(t)}$, а целый мини-батч размера $m$ таких векторов $X_{(t)}$, соответственно все дальнейшие размышления мы уже производим в матричном виде:

$$ Y_{(t)} = \phi(X_{(t)}^T \cdot W_x + Y_{(t-1)}^T \cdot W_y + b) = \phi([X_{(t)} Y_{(t-1)}] \cdot W + b) $$
где
$$ W = [W_x W_y]^T $$

*Операция в скобках квадратных -- конкатенация матриц

По размерностям:
* $Y_{(t)}$ -- матрица [$m$ x n_neurons]
* $X_{(t)}$ -- матрица [$m$ x n_features]
* $b$ -- вектор длины n_neurons
* $W_x$ -- веса между входами и нейронами размерностью [n_features x n_neurons]
* $W_y$ -- веса связей с прошлым выходом размерностью [n_neurons x n_neurons]

# RNN from scratch

### Задание

Напишите класс RNN реализующий vanilla RNN

In [2]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.W = nn.Linear(input_size + hidden_size, hidden_size)
        self.T = nn.Tanh()
        # batch_size * (input_size + hidden_size)
        #              (input_size + hidden_size) * hidden_size

    def forward(self, input_data, hidden):
#         print(input_data.shape, hidden.shape)
#         print(torch.cat([input_data, hidden], 1).shape)
        A = self.W.forward(torch.cat([input_data, hidden], 1))
        hidden = self.T(A)
        return hidden

    def initHidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)

In [3]:
input_feature_size = 6
hidden_size=5
batch_size=1

In [4]:
rnn = RNN(input_size=input_feature_size, hidden_size=hidden_size)
initial_hidden = rnn.initHidden(batch_size)

In [5]:
input_example = torch.rand([batch_size, input_feature_size])
new_hidden = rnn(input_example, initial_hidden)

In [6]:
print(new_hidden.shape)

torch.Size([1, 5])


In [7]:
print("initial_hidden: ", initial_hidden.numpy())
print("new_hidden: ", new_hidden.detach().numpy())

initial_hidden:  [[0. 0. 0. 0. 0.]]
new_hidden:  [[ 0.19985835  0.16307105 -0.00350405 -0.05697255  0.2615539 ]]


In [8]:
new_hidden = rnn(input_example, new_hidden)
print("new_hidden: ", new_hidden.detach().numpy())

new_hidden:  [[0.18390346 0.20857358 0.08123346 0.04525935 0.2951674 ]]


### Задание

Модифицировать код так, чтобы на входе можно было подавать батчи размером больше 1.

# Классификация картинок с RNN

<img src="https://cdn-images-1.medium.com/max/2000/1*wFYZpxTTiXVqncOLQd_CIQ.jpeg" width="800">

In [9]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


Загружаем данные

In [10]:
import torchvision
import torchvision.transforms as transforms

BATCH_SIZE = 64

# переводим все в тензоры
transform = transforms.Compose(
    [transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

In [11]:
!ls -lh data/

total 8,0K
drwxr-xr-x 3 kikos kikos 4,0K мар 23 14:22 imdb
drwxr-xr-x 4 kikos kikos 4,0K мар 23 13:57 MNIST


Populating the interactive namespace from numpy and matplotlib


In [12]:
%pylab inline
import numpy as np

# functions to show an image
def imshow(img):
    #img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))

## Как выглядит классификация с RNN в общем виде 

<img src="https://cdn-images-1.medium.com/max/1600/1*vhAfRLlaeOXZ-bruv7Ostg.png" width="400">

In [13]:
class ImageRNN(nn.Module):
    def __init__(self, batch_size, n_steps, n_inputs, n_neurons, n_outputs):
        super(ImageRNN, self).__init__()
        
        self.n_neurons = n_neurons
        self.batch_size = batch_size
        self.n_steps = n_steps
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        
        self.basic_rnn = nn.RNN(self.n_inputs, self.n_neurons) 
        
        self.FC = nn.Linear(self.n_neurons, self.n_outputs)
        
    def init_hidden(self,):
        # (num_layers, batch_size, n_neurons)
        return (torch.zeros(1, self.batch_size, self.n_neurons))
        
    def forward(self, X):
        # transforms X to dimensions: n_steps X batch_size X n_inputs
        X = X.permute(1, 0, 2) 
        
        self.batch_size = X.size(1)
        self.hidden = self.init_hidden()
        
        lstm_out, self.hidden = self.basic_rnn(X, self.hidden)      
        out = self.FC(self.hidden)
        
        return out.view(-1, self.n_outputs) # batch_size X n_output

In [14]:
N_STEPS = 28
N_INPUTS = 28
N_NEURONS = 150
N_OUTPUTS = 10
N_EPHOCS = 10

In [15]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
model = ImageRNN(BATCH_SIZE, N_STEPS, N_INPUTS, N_NEURONS, N_OUTPUTS)
logits = model(images.view(-1, 28,28))
print(logits[0:10])

tensor([[-0.0574, -0.0166, -0.0395,  0.0174,  0.0266,  0.0328, -0.0649, -0.0342,
         -0.1731, -0.0519],
        [-0.0555, -0.0030, -0.0335,  0.0162,  0.0292,  0.0346, -0.0583, -0.0344,
         -0.1745, -0.0510],
        [-0.0593,  0.0093, -0.0469,  0.0195,  0.0267,  0.0306, -0.0611, -0.0300,
         -0.1875, -0.0602],
        [-0.0579, -0.0103, -0.0470,  0.0060,  0.0229,  0.0339, -0.0607, -0.0399,
         -0.1641, -0.0461],
        [-0.0554, -0.0242, -0.0373,  0.0188,  0.0265,  0.0182, -0.0567, -0.0562,
         -0.1696, -0.0460],
        [-0.0576, -0.0136, -0.0335,  0.0175,  0.0242,  0.0273, -0.0586, -0.0358,
         -0.1790, -0.0541],
        [-0.0651, -0.0011, -0.0397, -0.0044,  0.0365,  0.0351, -0.0601, -0.0618,
         -0.1657, -0.0448],
        [-0.0560, -0.0068, -0.0366,  0.0236,  0.0300,  0.0302, -0.0684, -0.0346,
         -0.1731, -0.0622],
        [-0.0633, -0.0127, -0.0455,  0.0163,  0.0369,  0.0166, -0.0641, -0.0418,
         -0.1604, -0.0608],
        [-0.0675, -

## Обучаем

In [16]:
import torch.optim as optim

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Model instance
model = ImageRNN(BATCH_SIZE, N_STEPS, N_INPUTS, N_NEURONS, N_OUTPUTS)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def get_accuracy(logit, target, batch_size):
    ''' Obtain accuracy for training round '''
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()

In [17]:
for epoch in range(N_EPHOCS):  # loop over the dataset multiple times
    train_running_loss = 0.0
    train_acc = 0.0
    model.train()
    
    # TRAINING ROUND
    for i, data in enumerate(trainloader):
         # zero the parameter gradients
        optimizer.zero_grad()
        
        # reset hidden states
        model.hidden = model.init_hidden() 
        
        # get the inputs
        inputs, labels = data
        inputs = inputs.view(-1, 28,28) 

        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_running_loss += loss.detach().item()
        train_acc += get_accuracy(outputs, labels, BATCH_SIZE)
         
    model.eval()
    print('Epoch:  %d | Loss: %.4f | Train Accuracy: %.2f' 
          %(epoch, train_running_loss / i, train_acc/i))

Epoch:  0 | Loss: 0.8794 | Train Accuracy: 70.11
Epoch:  1 | Loss: 0.3838 | Train Accuracy: 88.18
Epoch:  2 | Loss: 0.2722 | Train Accuracy: 91.62
Epoch:  3 | Loss: 0.2240 | Train Accuracy: 93.15
Epoch:  4 | Loss: 0.1907 | Train Accuracy: 94.06
Epoch:  5 | Loss: 0.1692 | Train Accuracy: 94.78
Epoch:  6 | Loss: 0.1528 | Train Accuracy: 95.13
Epoch:  7 | Loss: 0.1466 | Train Accuracy: 95.35
Epoch:  8 | Loss: 0.1314 | Train Accuracy: 95.75
Epoch:  9 | Loss: 0.1235 | Train Accuracy: 95.96


### Смотрим что на тесте

In [17]:
test_acc = 0.0
for i, data in enumerate(testloader, 0):
    inputs, labels = data
    inputs = inputs.view(-1, 28, 28)

    outputs = model(inputs)

    test_acc += get_accuracy(outputs, labels, BATCH_SIZE)
        
print('Test Accuracy: %.2f'%( test_acc/i))

Test Accuracy: 9.13


# Сентимент анализ по аналогии

пишем сами с нуля

<img src="https://github.com/bentrevett/pytorch-sentiment-analysis/raw/bf8cc46e4823ebf9af721b595501ad6231c73632/assets/sentiment1.png">

$~ pip install torchtext

$~ python -m spacy download en


In [18]:
import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

In [19]:
from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL, root="./data")

In [20]:
ls -lh data/imdb/aclImdb/

total 1,7M
-rw-r--r-- 1 kikos kikos 882K июн 12  2011 imdbEr.txt
-rw-r--r-- 1 kikos kikos 827K апр 12  2011 imdb.vocab
-rw-r--r-- 1 kikos kikos 4,0K июн 26  2011 README
drwxr-xr-x 4 kikos kikos 4,0K апр 12  2011 test/
drwxr-xr-x 5 kikos kikos 4,0K июн 26  2011 train/


In [21]:
print(len(train_data))
print(len(test_data))

25000
25000


In [22]:
print(vars(train_data.examples[0]))

{'text': ['It', "'s", 'a', 'shame', 'that', 'Deliverance', 'is', 'mainly', 'known', 'as', 'the', 'redneck', 'rape', 'movie', 'and', 'for', 'Dueling', 'Banjos', '.', 'Even', 'people', 'that', 'have', 'seen', 'the', 'film', 'ca', "n't", 'get', 'their', 'mind', 'off', 'of', 'that', 'rape', 'scene', '.', 'It', "'s", 'not', 'as', 'bad', 'as', 'the', 'rape', 'scene', 'in', 'Pulp', 'Fiction', '.', 'It', "'s", 'certainly', 'not', 'as', 'bad', 'as', 'any', 'female', 'rape', 'scene', 'in', 'just', 'about', 'any', 'movie', '.', 'People', 'tend', 'to', 'miss', 'the', 'power', 'of', 'the', 'film', 'that', 'contains', 'the', 'infamous', 'buggery', 'scene.<br', '/><br', '/>The', 'acting', ',', 'plot', ',', 'cinematography', ',', 'and', 'soundtrack', 'of', 'Deliverance', 'all', 'lend', 'a', 'hand', 'to', 'it', "'s", 'brooding', 'charisma', '.', 'The', 'backcountry', 'it', 'was', 'shot', 'in', 'is', 'beautiful', 'and', 'is', 'quite', 'in', 'contrast', 'to', 'the', 'dark', 'subject', 'matter', '.', 'The

In [23]:
# Сделаем еще eval
import random

train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [24]:
# Сделаем словарь
TEXT.build_vocab(train_data, max_size=25000)
LABEL.build_vocab(train_data)

In [25]:
print(len(TEXT.vocab))
print(len(LABEL.vocab))

25002
2


In [26]:
vars(LABEL.vocab)

{'freqs': Counter({'neg': 8810, 'pos': 8690}),
 'itos': ['neg', 'pos'],
 'stoi': defaultdict(<function torchtext.vocab._default_unk_index()>,
             {'neg': 0, 'pos': 1}),
 'vectors': None}

Почему 25002, а не 25000?
Потому что $<unk>$ и $<pad>$

<img src="https://github.com/bentrevett/pytorch-sentiment-analysis/raw/bf8cc46e4823ebf9af721b595501ad6231c73632/assets/sentiment6.png" width="160">

In [27]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 202165), (',', 191769), ('.', 165291), ('and', 109267), ('a', 109187), ('of', 100767), ('to', 93592), ('is', 76329), ('in', 61186), ('I', 54838), ('it', 53748), ('that', 49252), ('"', 44622), ("'s", 43189), ('this', 42497), ('-', 36931), ('/><br', 35279), ('was', 35271), ('as', 30667), ('movie', 29983)]


* stoi (string to int)
* itos (int to string)

In [28]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [29]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fad30bafd08>, {'pos': 1, 'neg': 0})


In [74]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# собираем батчи так, чтобы в каждом батче были примеры наиболее похожей длины
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    device=device)

## Делаем модель

<img src="https://github.com/bentrevett/pytorch-sentiment-analysis/raw/bf8cc46e4823ebf9af721b595501ad6231c73632/assets/sentiment7.png" width="450">

* В эмбеддер (emb = [torch.nn.Embedding(num_embeddings, embedding_dim)](https://pytorch.org/docs/stable/nn.html?highlight=embedding#torch.nn.Embedding)) запихиваем тензор размерностью **[sentence length, batch size]**
* Эмбеддер возвращает тензор размерностью **[sentence length, batch size, embedding dim]**
* RNN (torch.nn.RNN(embedding_dim, hidden_dim)) возвращает 2 тензора, *output* размера [sentence length, batch size, hidden dim] и *hidden* размера [1, batch size, hidden dim]

In [78]:
def out(t):
    for x in t:
        print(TEXT.vocab.itos[x], end=' ')
    print()

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.input_dim = input_dim
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.embedding = nn.Embedding(len(TEXT.vocab), embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, dropout=0.6)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        # можно добавить линейный слой, который делает проекцию в 2 класса
    def forward(self, text):

        #text = [sent len, batch size]
#         print(text.shape)
        text = text.permute(1, 0)
#         print(text.shape)
#         out(text[0]);
#         out(text[1]);
        embedded = self.embedding(text)
        embedded = embedded.permute(1, 0, 2)
        #embedded = [sent len, batch size, emb dim]
        hidden = torch.zeros(1, BATCH_SIZE, self.hidden_dim)
#         print(embedded.shape)
#         print(hidden.shape)
        output, hidden = self.rnn(embedded, hidden)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
#         assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [79]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

N_EPHOCS = 25
N_WORDS = 40
N_EMBEDDED = 42
N_HIDDEN = 20
N_OUTPUT = 2
model = RNN(N_WORDS, N_EMBEDDED, N_HIDDEN, N_OUTPUT)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
def get_accuracy(logit, target, batch_size):    
    corrects = np.isclose(np.argmax(logit.detach().numpy(), axis=1), target.data.detach().numpy())
#     print(corrects)
    accuracy = 100.0 * corrects.sum()/batch_size
    return accuracy.item()

/home/kikos/anaconda3/lib/python3.5/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.6 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [80]:
for epoch in range(N_EPHOCS):  # loop over the dataset multiple times
    train_running_loss = 0.0
    train_acc = 0.0
    model.train()
    
    # TRAINING ROUND
    rounds = 0
    for i, _data in enumerate(train_iterator):
        if len(_data) != BATCH_SIZE:
            continue
        rounds += 1
        text = _data.text.clone()
#         print(text.shape)
        label = _data.label
        text = text.permute(1, 0)
#         print(text.shape)
        for x in text:
            while len(x) < N_WORDS:
                x.append(1)
        text = text[:,:N_WORDS]
        text = text.permute(1, 0)
         # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = model(text)
        res_label = torch.zeros(label.shape, dtype=torch.int64)
        for i in range(len(res_label)):
            if label[i] > 0.5:
                res_label[i] = 1
        label = res_label
        
        
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()

        train_running_loss += loss.detach().item()
        train_acc += get_accuracy(outputs, label, BATCH_SIZE)
        
    model.eval()
    print('Epoch:  %d | Loss: %.4f | Train Accuracy: %.2f' 
          %(epoch, train_running_loss / rounds, train_acc/rounds))

Epoch:  0 | Loss: 0.7067 | Train Accuracy: 50.44
Epoch:  1 | Loss: 0.6905 | Train Accuracy: 52.98
Epoch:  2 | Loss: 0.6840 | Train Accuracy: 55.31
Epoch:  3 | Loss: 0.6723 | Train Accuracy: 58.41
Epoch:  4 | Loss: 0.6455 | Train Accuracy: 62.63
Epoch:  5 | Loss: 0.5923 | Train Accuracy: 68.74
Epoch:  6 | Loss: 0.5226 | Train Accuracy: 74.66
Epoch:  7 | Loss: 0.4593 | Train Accuracy: 79.12
Epoch:  8 | Loss: 0.4063 | Train Accuracy: 82.39
Epoch:  9 | Loss: 0.3531 | Train Accuracy: 85.45
Epoch:  10 | Loss: 0.3039 | Train Accuracy: 88.18
Epoch:  11 | Loss: 0.2600 | Train Accuracy: 90.32
Epoch:  12 | Loss: 0.2262 | Train Accuracy: 91.88
Epoch:  13 | Loss: 0.1981 | Train Accuracy: 92.95
Epoch:  14 | Loss: 0.1738 | Train Accuracy: 93.92
Epoch:  15 | Loss: 0.1519 | Train Accuracy: 94.95
Epoch:  16 | Loss: 0.1278 | Train Accuracy: 95.96


KeyboardInterrupt: 

In [81]:
model.eval()
rounds = 0
running_loss = 0.0
running_accuracy = 0.0
for _data in valid_iterator:
    if len(_data) != BATCH_SIZE:
            continue
    rounds += 1
    text = _data.text.clone()
#         print(text.shape)
    label = _data.label
    text = text.permute(1, 0)
#         print(text.shape)
    for x in text:
        while len(x) < N_WORDS:
            x.append(1)
    text = text[:,:N_WORDS]
    text = text.permute(1, 0)
     # zero the parameter gradients
    # forward + backward + optimize
    outputs = model(text)
    res_label = torch.zeros(label.shape, dtype=torch.int64)
    for i in range(len(res_label)):
        if label[i] > 0.5:
            res_label[i] = 1
    label = res_label
    loss = criterion(outputs, label)
    
    running_loss += loss.detach().item()
    running_accuracy += get_accuracy(outputs, label, BATCH_SIZE)
print('Epoch:  %d | Loss: %.4f | Train Accuracy: %.2f' 
          %(epoch, running_loss / rounds, running_accuracy/rounds))

Epoch:  17 | Loss: 1.0789 | Train Accuracy: 69.07
